# Suggest: классификация категорий по названиям объявлений товаров Авито

**Цель:** предсказать `category_id` по тексту `title` карточек товаров маркетплейса.

**Данные:** `suggest_train.csv` (title, category_id) и `suggest_test.csv`.

**Метрики:** accuracy и accuracy@3.

### Импорты

In [1]:
import re

import nltk
import numpy as np
import pandas as pd
import pymorphy3
from gensim.models import FastText
from gensim.models.word2vec import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Предобработка данных

In [2]:
train = pd.read_csv("suggest_train.csv")
test = pd.read_csv("suggest_test.csv")

In [3]:
train.head()

,item_id,title,category_id,name
0,335330,Раствор. Товарный бетон от производителя,15,Для дома и дачи|Ремонт и строительство|Стройма...
1,137480,Детская кроватка,16,"Для дома и дачи|Мебель и интерьер|Кровати, див..."
2,242690,"Кровать -качалка с ящиком для вещей, два полож...",34,Личные вещи|Товары для детей и игрушки|Детская...
3,434016,Линолеум комитекс лин печора орфей 431,15,Для дома и дачи|Ремонт и строительство|Стройма...
4,237890,Распродажа имущества предприятия. Компьютеры,7,Бытовая электроника|Настольные компьютеры


In [103]:
most_common_category = train.groupby("title")["category_id"].agg(lambda x: x.mode()[0]).reset_index().rename(columns={"category_id": "most_common_category_id"})
train = train.merge(most_common_category, on="title", how="left")
train["category_id"] = train["most_common_category_id"]
train = train.drop(columns=["most_common_category_id", "item_id", "name"])

In [104]:
nltk.download("punkt")
nltk.download("stopwords")

morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words("russian"))
stop_words.update(stopwords.words("english"))
english_stemmer = SnowballStemmer("english")


def lemmatize_token(token):
    if re.match(r"^[a-zA-Z]+$", token):  # английское слово
        return english_stemmer.stem(token)
    else:  # русское слово
        parsed = morph.parse(token)[0]
        return parsed.normal_form


def preprocess_text(text, base=False):
    # 1. Приведение к нижнему регистру
    text = text.lower()

    # 2. Удаление небуквенных символов (кириллица + латиница + пробелы)
    text = re.sub(r"[^а-яА-ЯёЁa-zA-Z0-9\\s]", " ", text)

    # 3. Токенизация
    tokens = text.split()

    # 4. Удаление стоп‑слов и коротких токенов
    tokens = [t for t in tokens if t not in stop_words and len(t) > 1]

    if base:
        return " ".join(tokens)

    # 5. Лемматизация (для русских) + стемминг (для английских)
    tokens = [lemmatize_token(t) for t in tokens]

    return " ".join(tokens)

[nltk_data] Downloading package punkt to /home/qqml/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/qqml/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [105]:
train["title_clean"] = train["title"].apply(lambda text: preprocess_text(text))
train.drop(axis=0, index=train.loc[train["title_clean"] == ""].index, inplace=True)  # уберем пустые после очистки строки чтобы они не прошли дальше

In [106]:
data = pd.concat([train, test], axis=0)
all_data = data["title"]

In [107]:
X_test = test["title"]
X_train, X_val, y_train, y_val = train_test_split(train["title"], train["category_id"], test_size=0.1, stratify=train["category_id"])

In [108]:
unlabeled_data = [preprocess_text(text, base=True) for text in all_data]  # данные без лемматизации/стемминга будут использоваться для FastText
unlabeled_data_clean = [preprocess_text(text) for text in all_data]  # а чистые данные лучше подойдут для TF-IDF и W2V
X_train_clean = [preprocess_text(text) for text in X_train]
X_val_clean = [preprocess_text(text) for text in X_val]
X_test_clean = [preprocess_text(text) for text in X_test]

In [109]:
def top3_accuracy_score(y_true, probas):
    y_pred = top3_answer(probas)
    return sum(1 for i in range(len(y_true)) if y_true[i] in y_pred[i]) / len(y_true)


def top3_answer(probas):
    return np.argsort(probas, axis=1)[:, ::-1][:, :3]

In [110]:
WORD_PATTERN = "(?u)\\b\\w\\w+\\b"  # паттерн для токенизации из sklearn
reg_exp = re.compile(pattern=WORD_PATTERN)
sentences = [reg_exp.findall(s.lower()) for s in unlabeled_data]
sentences_clean = [reg_exp.findall(s.lower()) for s in unlabeled_data_clean]

In [111]:
class Word2VecTransformer:
    def __init__(self, w2v_model, word_pattern):
        self.w2v_model = w2v_model
        self.word_pattern = word_pattern
        self.re = re.compile(pattern=self.word_pattern)

    def fit(self, X):
        return self

    def transform(self, X):
        X_transformed = np.zeros((len(X), self.w2v_model.wv.vector_size))
        for i, title in enumerate(X):
            tokens = self.re.findall(title.lower())
            vectors = []
            for token in tokens:
                if token in self.w2v_model.wv.key_to_index:
                    vectors.append(self.w2v_model.wv.get_vector(token))

            if vectors:
                X_transformed[i] = np.mean(vectors, axis=0)
            else:
                X_transformed[i] = np.zeros((self.w2v_model.wv.vector_size))

        return X_transformed

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

## TF_IDF

In [112]:
tfidf_chars_params = {
    "analyzer": "char_wb",
    "ngram_range": (3, 6),
    "max_features": 60000,
    "min_df": 1,
    "max_df": 0.9,
    "lowercase": True,
    "use_idf": True,
    "sublinear_tf": True,
    "norm": "l2",
}


tfidf_chars = TfidfVectorizer(**tfidf_chars_params)
X_train_tfidf_chars = tfidf_chars.fit_transform(X_train_clean)
X_val_tfidf_chars = tfidf_chars.transform(X_val_clean)

In [113]:
lr_tfidf = LogisticRegression(solver="newton-cg", C=5, class_weight="balanced", tol=1e-4, max_iter=2000)
lr_tfidf.fit(X_train_tfidf_chars, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",5
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:

In [114]:
probas = lr_tfidf.predict_proba(X_val_tfidf_chars)
preds = lr_tfidf.predict(X_val_tfidf_chars)
print(f"accuracy: {accuracy_score(y_val, preds)} accuracy@3: {top3_accuracy_score(y_val.values, probas)}")

accuracy: 0.789 accuracy@3: 0.909


In [115]:
y_pred_proba = lr_tfidf.predict_proba(X_val_tfidf_chars)
top_3_preds = np.argsort(y_pred_proba, axis=1)[:, -3:]
top_3_probas = np.sort(y_pred_proba, axis=1)[:, -3:]

hits = np.any(top_3_preds == y_val.values[:, np.newaxis], axis=1)
errors_mask = ~hits

X_val_clean_series = pd.Series(X_val_clean, index=y_val.index)

error_df = pd.DataFrame(
    {
        "text": X_val_clean_series[errors_mask],
        "true_category": y_val[errors_mask],
        "true_category_proba": y_pred_proba[errors_mask, y_val[errors_mask]],
        "top1_pred": top_3_preds[errors_mask, -1],
        "top1_proba": top_3_probas[errors_mask, -1],
        "top2_pred": top_3_preds[errors_mask, -2],
        "top2_proba": top_3_probas[errors_mask, -2],
        "top3_pred": top_3_preds[errors_mask, -3],
        "top3_proba": top_3_probas[errors_mask, -3],
    }
)

print(f"Всего ошибок: {len(error_df)}")
display(error_df.head(20))

Всего ошибок: 182


,text,true_category,true_category_proba,top1_pred,top1_proba,top2_pred,top2_proba,top3_pred,top3_proba
8949,sky watchrr dob,47,0.015325,36,0.324855,6,0.135861,8,0.100063
11508,комплект мебель мадрид,16,0.040029,22,0.294199,18,0.172906,23,0.165892
12540,бусы рог кость старинный,40,0.015340,44,0.406606,23,0.141285,19,0.124497
11549,samsung sm t310,3,0.084934,2,0.485767,1,0.161082,5,0.093952
3306,январь цена 2015 год,25,0.011796,45,0.430120,48,0.058983,32,0.045899
9757,мандарин,24,0.033642,43,0.147671,44,0.068924,37,0.052827
9490,фунгицид против грибной бактериальный заболеван,24,0.017578,15,0.280595,37,0.105080,21,0.043167
8829,кпб сатина романтика париж подарочный упаковка,23,0.041867,27,0.367168,43,0.137977,30,0.044235
11481,акупунктура 3x50мма,37,0.016041,53,0.073803,15,0.068579,49,0.055807
3214,звонкий чёрный хороший чехол,50,0.008237,9,0.486821,27,0.061800,43,0.036321


## Word2Vec

In [116]:
w2v = Word2Vec(
    sentences_clean,
    epochs=200,
    min_count=1,
    sg=1,
    window=3,
    vector_size=300,
    workers=12,
)

In [117]:
w2v_transformer = Word2VecTransformer(w2v, WORD_PATTERN)
X_train_w2v = w2v_transformer.fit_transform(X_train_clean)
X_val_w2v = w2v_transformer.transform(X_val_clean)

In [118]:
lr_v2w = LogisticRegression(solver="newton-cg", class_weight="balanced", tol=1e-4, max_iter=2000)
lr_v2w.fit(X_train_w2v, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [119]:
probas = lr_v2w.predict_proba(X_val_w2v)
preds = lr_v2w.predict(X_val_w2v)
print(f"accuracy: {accuracy_score(y_val, preds)} accuracy@3: {top3_accuracy_score(y_val.values, probas)}")

accuracy: 0.772 accuracy@3: 0.8835


Попробуем соединить TF_IDF и W2V

In [120]:
X_train_tfidf_w2v = hstack([X_train_w2v, X_train_tfidf_chars]).tocsr().astype(np.float32)
X_val_tfidf_w2v = hstack([X_val_w2v, X_val_tfidf_chars]).tocsr().astype(np.float32)

In [ ]:
lr_tfidf_v2w = LogisticRegression(solver="newton-cg", class_weight="balanced", tol=1e-4, max_iter=2000)
lr_tfidf_v2w.fit(X_train_tfidf_w2v, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",5
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:

In [134]:
probas = lr_tfidf_v2w.predict_proba(X_val_tfidf_w2v)
preds = lr_tfidf_v2w.predict(X_val_tfidf_w2v)
print(f"accuracy: {accuracy_score(y_val, preds)} accuracy@3: {top3_accuracy_score(y_val.values, probas)}")

accuracy: 0.779 accuracy@3: 0.8955


## FastText

In [123]:
ft = FastText(
    sentences,
    vector_size=300,
    window=3,
    min_count=1,
    sg=1,
    epochs=250,
    min_n=3,
    max_n=5,
    hs=0,
    negative=10,
    workers=12,
)

In [124]:
ft_transformer = Word2VecTransformer(ft, WORD_PATTERN)
X_train_ft = ft_transformer.transform(X_train)
X_val_ft = ft_transformer.transform(X_val)

In [125]:
lr_ft = LogisticRegression(solver="newton-cg", class_weight="balanced", tol=1e-4, max_iter=2000)
lr_ft.fit(X_train_ft, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [126]:
probas = lr_ft.predict_proba(X_val_ft)
preds = lr_ft.predict(X_val_ft)
print(f"accuracy: {accuracy_score(y_val, preds)} accuracy@3: {top3_accuracy_score(y_val.values, probas)}")

accuracy: 0.757 accuracy@3: 0.88


Добавим к FastText TF-IDF

In [127]:
X_train_ft_sparse = csr_matrix(X_train_ft)
X_val_ft_sparse = csr_matrix(X_val_ft)

X_train_ft_tfidf = hstack([X_train_ft_sparse, X_train_tfidf_chars]).tocsr().astype(np.float32)
X_val_ft_tfidf = hstack([X_val_ft_sparse, X_val_tfidf_chars]).tocsr().astype(np.float32)

In [128]:
lr_tfidf_ft = LogisticRegression(solver="newton-cg", class_weight="balanced", tol=1e-4, max_iter=2000)
lr_tfidf_ft.fit(X_train_ft_tfidf, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [129]:
probas = lr_tfidf_ft.predict_proba(X_val_ft_tfidf)
preds = lr_tfidf_ft.predict(X_val_ft_tfidf)
print(f"accuracy: {accuracy_score(y_val, preds)} accuracy@3: {top3_accuracy_score(y_val.values, probas)}")

accuracy: 0.768 accuracy@3: 0.891


Наилучшее качество показал TF-IDF подход. Добавление каких-то продвинутых технологий преобразования текста в векторы не дало значительного улучшения скорее всего из-за недостаточного количества данных для обучения

In [130]:
X_test_tfidf_chars = tfidf_chars.transform(X_test_clean)

result = top3_answer(lr_tfidf.predict_proba(X_test_tfidf_chars))

In [131]:
submission_df = pd.DataFrame(result, columns=["top1", "top2", "top3"])

In [132]:
submission_df.to_csv("solution.csv", header=True, index=False)